In [1]:
from astral import LocationInfo
from astral.geocoder import database, lookup
from astral.sun import sun
from datetime import datetime, date, time, timedelta
from dateutil import tz
import dtale
import holidays
import numpy as np
import pandas as pd
import requests

pd.options.display.max_rows = 100
pd.options.display.max_columns = 0

# Display all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Purpose

Create DataFrames for each dimension and fact table

In [2]:
# Paths to data
denver_data_path = '../../data/denver_crime_data/'
vancouver_data_path = '../../data/vancouver_crime_data/'
dimensions_path = '../../data/dimensional_model/'

# Load all crimes
all_crimes = pd.read_csv('../../data/all_crimes.csv')

# Reset datetimes
all_crimes.FIRST_OCCURRENCE_DATE = pd.DatetimeIndex(all_crimes.FIRST_OCCURRENCE_DATE)
all_crimes.FIRST_OCCURRENCE_TIME = pd.to_datetime(all_crimes.FIRST_OCCURRENCE_TIME).dt.time

D:\ProgramData\Anaconda3\envs\crime-data-mart\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,8,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Date Dimension

In [3]:
# Create date dim with all dates from earliest crime date to current date

# Date dim date ranges
start_date = all_crimes.FIRST_OCCURRENCE_DATE.min()
end_date = pd.to_datetime(datetime.now())
dates = pd.date_range(start = start_date, end = end_date)

# Holiday information
can_holidays = holidays.Canada()
us_holidays = holidays.UnitedStates()

# Get some date attributes
days = [date.day for date in dates]
months = [date.month for date in dates]
years = [date.year for date in dates]
day_of_years = [date.dayofyear for date in dates]
week_of_years = [date.weekofyear for date in dates]
weekdays = [date.weekday() + 1 for date in dates]
is_weekends = [weekday in {6,7} for weekday in weekdays]
quarters = [date.quarter for date in dates]
month_starts = [date.is_month_start for date in dates]
month_ends = [date.is_month_end for date in dates]
year_starts = [date.is_year_start for date in dates]
year_ends = [date.is_year_end for date in dates]

# Get holidays
date_can_holidays = [can_holidays.get(date) for date in dates]
date_is_can_holidays = [not holiday is None for holiday in date_can_holidays]
date_us_holidays = [us_holidays.get(date) for date in dates]
date_is_us_holidays = [not holiday is None for holiday in date_us_holidays]

# Data dimension
date_dim = pd.DataFrame(
    {
        'FULL_DATE': dates,
        'DAY': days,
        'MONTH': months,
        'YEAR': years,
        'DAY_OF_YEAR': day_of_years,
        'WEEK_OF_YEAR': week_of_years,
        'WEEKDAY': weekdays,
        'IS_WEEKEND': is_weekends,
        'QUARTER': quarters,
        'IS_MONTH_START': month_starts,
        'IS_MONTH_END': month_ends,
        'IS_YEAR_START': year_starts,
        'IS_YEAR_END': year_ends,
        'IS_CAN_HOLIDAY': date_is_can_holidays,
        'CAN_HOLIDAY_NAME': date_can_holidays,
        'IS_US_HOLIDAY': date_is_us_holidays,
        'US_HOLIDAY_NAME': date_us_holidays
    }
)

# Add surrogate date key as just date YYYYMMDD (From Kimball textbook)
date_dim['DATE_PK'] = date_dim.FULL_DATE.dt.strftime('%Y%m%d').apply(int)

date_dim.shape
date_dim.head(10)

(1878, 18)

,FULL_DATE,DAY,MONTH,YEAR,DAY_OF_YEAR,WEEK_OF_YEAR,WEEKDAY,IS_WEEKEND,QUARTER,IS_MONTH_START,IS_MONTH_END,IS_YEAR_START,IS_YEAR_END,IS_CAN_HOLIDAY,CAN_HOLIDAY_NAME,IS_US_HOLIDAY,US_HOLIDAY_NAME,DATE_PK
0,2015-01-02,2,1,2015,2,1,5,False,1,False,False,False,False,False,None,False,None,20150102
1,2015-01-03,3,1,2015,3,1,6,True,1,False,False,False,False,False,None,False,None,20150103
2,2015-01-04,4,1,2015,4,1,7,True,1,False,False,False,False,False,None,False,None,20150104
3,2015-01-05,5,1,2015,5,2,1,False,1,False,False,False,False,False,None,False,None,20150105
4,2015-01-06,6,1,2015,6,2,2,False,1,False,False,False,False,False,None,False,None,20150106
5,2015-01-07,7,1,2015,7,2,3,False,1,False,False,False,False,False,None,False,None,20150107
6,2015-01-08,8,1,2015,8,2,4,False,1,False,False,False,False,False,None,False,None,20150108
7,2015-01-09,9,1,2015,9,2,5,False,1,False,False,False,False,False,None,False,None,20150109
8,2015-01-10,10,1,2015,10,2,6,True,1,False,False,False,False,False,None,False,None,20150110
9,2015-01-11,11,1,2015,11,2,7,True,1,False,False,False,False,False,None,False,None,20150111


In [4]:
# Add surrogate key to all_crimes fact table
all_crimes = all_crimes.merge(date_dim[['DATE_PK', 'FULL_DATE']], how = 'left', left_on = 'FIRST_OCCURRENCE_DATE', right_on = 'FULL_DATE').drop('FULL_DATE', axis = 1)

# Save dimension
date_dim.to_csv(dimensions_path + 'date_dimension.csv', index = False)

# Location Dimension

No data could be found for yearly demographic data on the population for both Denver and Vancouver and for per neighborhood. So instead, we will just store the city and neighborhood population for one year  

- https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/prof/details/page.cfm?Lang=E&Geo1=CSD&Code1=5915022&Geo2=PR&Code2=01&Data=Count&SearchText=5915022&SearchType=Begins&SearchPR=01&B1=All&Custom=&TABID=3
- http://worldpopulationreview.com/us-cities/denver-population/

In [5]:
# Location dimension
location_cols = ['LOCATION', 'NEIGHBORHOOD', 'LATITUDE', 'LONGITUDE', 'CITY']
location_dim = all_crimes[location_cols].copy()
location_dim.drop_duplicates(keep = 'first', inplace = True)

# Add demographic data - Population and Crime Rate
all_crimes['YEAR'] = all_crimes.FIRST_OCCURRENCE_DATE.dt.year

# Add population
denver_population = 716492 # Copy pasted from site above

vancouver_demographics = pd.read_csv(vancouver_data_path + 'CensusProfile2016-ProfilRecensement2016-20200221013316.csv', encoding = 'ISO-8859-1', skiprows = 1)
vancouver_population = int(vancouver_demographics.Total[1])

city_populations = pd.DataFrame({'CITY': ['Denver', 'Vancouver'], 
                                 'CITY_POPULATION': [denver_population, vancouver_population],
                                 'YEAR': [2018, 2016]})
location_dim = location_dim.merge(city_populations, how = 'left', on = 'CITY')

# Get number of crimes for given city and year 
num_crimes = city_populations.merge(all_crimes, how = 'left', on = ['CITY', 'YEAR']).CITY.value_counts().rename_axis('CITY').to_frame('NUMBER_CRIMES')
location_dim = location_dim.merge(num_crimes, how = 'left', on = 'CITY')

# Calculate crime rate 
location_dim['CRIME_RATE'] = (location_dim.NUMBER_CRIMES / location_dim.CITY_POPULATION) * 1e5

# Drop temp columns
all_crimes.drop('YEAR', axis = 1, inplace = True)
location_dim.drop(['CITY_POPULATION', 'YEAR', 'NUMBER_CRIMES'], axis = 1, inplace = True)

# Add surrogate PK
location_dim['LOCATION_PK'] = np.arange(1, location_dim.shape[0] + 1)

location_dim.shape
location_dim.head(10)

(191530, 7)

,LOCATION,NEIGHBORHOOD,LATITUDE,LONGITUDE,CITY,CRIME_RATE,LOCATION_PK
0,10XX SITKA SQ,Fairview,49.266678,-123.129029,Vancouver,5995.38232,1
1,10XX ALBERNI ST,West End,49.285255,-123.123649,Vancouver,5995.38232,2
2,10XX ALBERNI ST,West End,49.284981,-123.123053,Vancouver,5995.38232,3
3,10XX ALBERNI ST,West End,49.284794,-123.122946,Vancouver,5995.38232,4
4,10XX ALBERNI ST,West End,49.284715,-123.122824,Vancouver,5995.38232,5
5,10XX ALBERNI ST,West End,49.284666,-123.122749,Vancouver,5995.38232,6
6,10XX ALBERNI ST,West End,49.284445,-123.122412,Vancouver,5995.38232,7
7,10XX ALBERNI ST,West End,49.284519,-123.122348,Vancouver,5995.38232,8
8,10XX ALBERNI ST,West End,49.284396,-123.122337,Vancouver,5995.38232,9
9,10XX BARCLAY ST,West End,49.282661,-123.126206,Vancouver,5995.38232,10


In [6]:
# Add surrogate key to crime fact table
all_crimes = all_crimes.merge(location_dim, how = 'left', on = location_cols)

# Save dimension
location_dim.to_csv(dimensions_path + 'location_dimension.csv', index = False)

# Crime Dimension

In [7]:
# Crime dimension
crime_cols = ['FIRST_OCCURRENCE_TIME', 'LAST_OCCURRENCE_TIME', 'REPORTED_TIME', 'OFFENSE_TYPE', 'OFFENSE_CATEGORY', 'IS_VIOLENT']
crime_dim = all_crimes[crime_cols].copy()
crime_dim.drop_duplicates(keep = 'first', inplace = True)

# Add surrogate PK
crime_dim['CRIME_PK'] = np.arange(1, crime_dim.shape[0] + 1)

crime_dim.shape
crime_dim.head(5)

(390561, 7)

,FIRST_OCCURRENCE_TIME,LAST_OCCURRENCE_TIME,REPORTED_TIME,OFFENSE_TYPE,OFFENSE_CATEGORY,IS_VIOLENT,CRIME_PK
0,02:06:00,NaN,NaN,Burglary of a business with forced entry,Burglary,False,1
1,04:12:00,NaN,NaN,Burglary of a business with forced entry,Burglary,False,2
2,20:00:00,NaN,NaN,Burglary of a business with forced entry,Burglary,False,3
3,06:17:00,NaN,NaN,Burglary of a business with forced entry,Burglary,False,4
4,20:53:00,NaN,NaN,Burglary of a business with forced entry,Burglary,False,5


In [8]:
# Add surrogate key to crime fact table
all_crimes = all_crimes.merge(crime_dim, how = 'left', on = crime_cols)

# Save dimension
crime_dim.to_csv(dimensions_path + 'crime_dimension.csv', index = False)

# Event Dimension

No API could be found that provided past events so we will just Hockey game events for now 
- https://gitlab.com/dword4/nhlapi
- https://statsapi.web.nhl.com/api/v1/teams

Team ids are Vancouver - 23, Denver - 21

In [9]:
# Get games for both cities from earliest crime date to now

start_date = str(date_dim.FULL_DATE.min().date())
end_date = str(datetime.now().date())

cities = pd.DataFrame({
    'ID': [23, 21],
    'CITY': ['Vancouver', 'Denver']
})

# Get request
url = 'https://statsapi.web.nhl.com/api/v1/schedule?'
params = {
    'startDate': start_date,
    'endDate': end_date,
}

# Add null row for no event
event_dim = pd.DataFrame({
    'DATE': [None],
    'LOCATION': [None],
    'CITY': [None],
    'NAME': ['No Event'],
    'TYPE': [None],
    'LOCATION_SIZE': [None]
})

# Get all games for Vancouver
for _, city in cities.iterrows():
    params['teamId'] = city.ID
    response = requests.get(url, params)
    response = response.json()['dates']

    dates = [pd.to_datetime(date['date']) for date in response]
    home_ids = [game['games'][0]['teams']['home']['team']['id'] for game in response]
    home_names = [game['games'][0]['teams']['home']['team']['name'] for game in response]
    away_names = [game['games'][0]['teams']['away']['team']['name'] for game in response]
    names = [f'{home_names[i]} VS {away_names[i]}' for i in range(0, len(home_names))]
    locations = [game['games'][0]['venue']['name'] for game in response]

    games = pd.DataFrame({
        'DATE': dates,
        'ID': home_ids,
        'LOCATION': locations,
        'CITY': [city.CITY] * len(dates),
        'TYPE': ['Hockey Game'] * len(dates),
        'NAME': names,
        'LOCATION_SIZE': [None] * len(dates)
    }).query(f'ID == {city.ID}').drop('ID', axis = 1)
    
    event_dim = pd.concat([event_dim, games], ignore_index = True)
    
# Add surrogate PK
event_dim['EVENT_PK'] = np.arange(1, event_dim.shape[0] + 1)

event_dim.shape
event_dim.head(5)

(484, 7)

,DATE,LOCATION,CITY,NAME,TYPE,LOCATION_SIZE,EVENT_PK
0,NaT,None,None,No Event,None,None,1
1,2015-01-03,Rogers Arena,Vancouver,Vancouver Canucks VS Detroit Red Wings,Hockey Game,None,2
2,2015-01-06,Rogers Arena,Vancouver,Vancouver Canucks VS New York Islanders,Hockey Game,None,3
3,2015-01-08,Rogers Arena,Vancouver,Vancouver Canucks VS Florida Panthers,Hockey Game,None,4
4,2015-01-10,Rogers Arena,Vancouver,Vancouver Canucks VS Calgary Flames,Hockey Game,None,5


In [10]:
# Add surrogate key to fact table
all_crimes = all_crimes.merge(event_dim.drop(['LOCATION', 'NAME'], axis = 1), how = 'left', left_on = ['FIRST_OCCURRENCE_DATE', 'CITY'], right_on = ['DATE', 'CITY'])
all_crimes.drop('DATE', axis = 1, inplace = True)
all_crimes.loc[all_crimes.EVENT_PK.isnull(), 'EVENT_PK'] = 1
all_crimes.EVENT_PK = all_crimes.EVENT_PK.apply(int)
event_dim.drop('DATE', axis = 1, inplace = True)

# Save dimension
event_dim.to_csv(dimensions_path + 'event_dimension.csv', index = False)

# Crime Fact Table

Now let's add all facts that aren't already in the crime data.  
To get the night time, we'll use Astral package since it takes into account the day of the year in calculating sun phases

In [11]:
# IS NIGHTTIME

# Denver sun phases
tz_mst = tz.gettz('MST')
denver = lookup('Denver', database())
suns = [sun(denver.observer, date = date.date(), tzinfo = tz_mst) for date in date_dim.FULL_DATE]

dusks = pd.Series([s['dusk'].time() for s in suns])
dawns = pd.Series([s['dawn'].time() for s in suns])

denver_nights = pd.DataFrame({
    'DATE': date_dim.FULL_DATE,
    'DUSK': dusks,
    'DAWN': dawns,
    'CITY': 'Denver'
})

# Vancouver sun phases
tz_mst = tz.gettz('PST')
denver = lookup('Vancouver', database())
suns = [sun(denver.observer, date = date.date(), tzinfo = tz_mst) for date in date_dim.FULL_DATE]

dusks = pd.Series([s['dusk'].time() for s in suns])
dawns = pd.Series([s['dawn'].time() for s in suns])

vancouver_nights = pd.DataFrame({
    'DATE': date_dim.FULL_DATE,
    'DUSK': dusks,
    'DAWN': dawns,
    'CITY': 'Vancouver'
})

# Combine
nights = pd.concat([denver_nights, vancouver_nights], ignore_index = True)

# Add dusks/dawn 
all_crimes = all_crimes.merge(nights, how = 'left', left_on = ['CITY', 'FIRST_OCCURRENCE_DATE'], right_on = ['CITY', 'DATE']).drop('DATE', axis = 1)

# Add nighttime status
all_crimes['IS_NIGHTTIME'] = (all_crimes.FIRST_OCCURRENCE_TIME > all_crimes.DUSK) | (all_crimes.FIRST_OCCURRENCE_TIME < all_crimes.DAWN)
all_crimes.loc[all_crimes.FIRST_OCCURRENCE_TIME.isnull(), 'IS_NIGHTTIME'] = pd.NA

# Drop unneeded columns
all_crimes.drop(['DUSK', 'DAWN'], axis = 1, inplace = True)

In [12]:
# Drop unneeded columns
cols_keep = ['IS_TRAFFIC', 'IS_FATAL', 'IS_NIGHTTIME', 'DATE_PK', 'CRIME_PK', 'LOCATION_PK', 'EVENT_PK']
all_crimes = all_crimes[cols_keep]

# Add surrogate PK
all_crimes['PK'] = np.arange(1, all_crimes.shape[0] + 1)

# Save facts
all_crimes.to_csv(dimensions_path + 'crimes_fact.csv', index = False)

all_crimes.head(5)

,IS_TRAFFIC,IS_FATAL,IS_NIGHTTIME,DATE_PK,CRIME_PK,LOCATION_PK,EVENT_PK,PK
0,False,False,True,20190307,1,1,1,1
1,False,False,True,20190827,2,2,1,2
2,False,False,False,20171114,3,3,1,3
3,False,False,True,20180302,4,4,156,4
4,False,False,True,20150204,5,5,1,5


In [13]:
all_crimes.isnull().mean() * 100

IS_TRAFFIC      0.000000
IS_FATAL        0.000000
IS_NIGHTTIME    2.531231
DATE_PK         0.000000
CRIME_PK        0.000000
LOCATION_PK     0.000000
EVENT_PK        0.000000
PK              0.000000
dtype: float64